# Simple 3D Finite Elasticity model

This is simple Elasticity Model in 3D. The model is implemented in Python using the FEniCS Project.

In [1]:
import numpy as np

import dolfinx

from mpi4py import MPI
from petsc4py import PETSc

from dolfinx import fem, mesh, io, plot, log,default_scalar_type
from dolfinx.fem import (Constant, dirichletbc, Function, functionspace, Expression )
from dolfinx.fem.petsc import NonlinearProblem,LinearProblem
from dolfinx.nls.petsc import NewtonSolver
from dolfinx.io import XDMFFile
import ufl
from ufl import (TestFunctions, TrialFunction, Identity, grad, det, div, dev, inv, tr, sqrt, conditional , gt, dx, inner, derivative, dot, ln, split,TestFunction,indices,as_tensor)
from basix.ufl import element, mixed_element
from datetime import datetime
from dolfinx.plot import vtk_mesh

import pyvista
pyvista.set_jupyter_backend('client')
## Define temporal parameters

## Some parameters

In [2]:
e_dot = 0.001 # strain rate per second 
e_max = 0.01 # absolute value of maximum strain


E = 200e9 # Young’s modulus 
Y0 = 100e6 # Initial yield 
h0 = 0 # Initial hardening modulus 
Y_star = 100e6 # Yield saturation 
r = 0 #Hardening exponent 
C = 0 #Backstress modulus 
gamma = 0 # Nonlinear kinematic parameter



# Define Geometry

In [3]:
length = 10.0 # mm
longside= 100
num_elements = 10
domain = mesh.create_box(MPI.COMM_WORLD,[[0.0,0.0,0.0],[longside,length,length]],[num_elements*10,num_elements,num_elements],mesh.CellType.tetrahedron)

In [4]:
plotter = pyvista.Plotter()
vtkdata = vtk_mesh(domain, domain.topology.dim)
grid = pyvista.UnstructuredGrid(*vtkdata)
actor = plotter.add_mesh(grid, show_edges=True)
plotter.show()
plotter.close()

Widget(value="<iframe src='http://localhost:37921/index.html?ui=P_0xffffa81afdc0_0&reconnect=auto' style='widt…

# Definig Function Spaces


In [5]:
# Tensor space
import basix.ufl

#T_Ele = basix.ufl.quadrature_element(domain.topology.cell_name(),value_shape=(3,3),degree=1)


n = 0

U_Ele = element("Lagrange", domain.basix_cell(),1,shape=(3,))

#T_quad = functionspace(domain,T_Ele)
V_u = functionspace(domain,U_Ele)

In [6]:
#MAT Properties

K = Constant(domain,160e9) #200 MPa Bulk Modulus
G = Constant(domain,80.0e9) #80 Mpa Shear Modulus

I = Identity(3)
II = ufl.outer(I,I)
i,j,k,l = indices(4)
Ident_sym_ii =1/2 * (I[i,k]*I[j,l] + I[i,l]*I[j,k])

Ident_sym = as_tensor(Ident_sym_ii,(i,j,k,l))
C = 2*G*Ident_sym + (K-(2/3)*G)*II

In [7]:
Y = 9*G.value*K.value/(G.value+3*K.value)

In [8]:
u = TrialFunction(V_u)
v = ufl.TestFunction(V_u)
E =ufl.sym(ufl.grad(u))
T = as_tensor(C[i,j,k,l]*E[k,l],(i,j))

In [9]:
T_2 = fem.Constant(domain, default_scalar_type((0, 0, 0)))
ds = ufl.Measure("ds", domain=domain)
a = inner(T , grad(v) )*dx
L =  ufl.dot(T_2, v) * ds

In [10]:
def clamped_boundary(x):
    return np.isclose(x[0], 0)


def strech(x):
    return np.isclose(x[0], longside)


fdim = domain.topology.dim - 1
boundary_facets = mesh.locate_entities_boundary(domain, fdim, clamped_boundary)
boundary_facets_2 = mesh.locate_entities_boundary(domain, fdim, strech)

u_D = np.array([0, 0, 0], dtype=default_scalar_type)
u_D2 = np.array(10, dtype=default_scalar_type)
bc = fem.dirichletbc(u_D, fem.locate_dofs_topological(V_u, fdim, boundary_facets), V_u)
bc2 = fem.dirichletbc(u_D2, fem.locate_dofs_topological(V_u.sub(0), fdim, boundary_facets_2), V_u.sub(0))

In [11]:
problem = LinearProblem(a, L, bcs=[bc,bc2], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

In [12]:
U = functionspace(domain,("DG",0))

mises = Function(U)
E =ufl.sym(ufl.grad(uh))
T_val = as_tensor(C[i,j,k,l]*E[k,l],(i,j))
T0   = T_val- (1/3)*tr(T_val)*Identity(3)
mises_val= sqrt(3/2*inner(T0,T0))
mises_expr = Expression(mises_val,U.element.interpolation_points())
mises.interpolate(mises_expr)


In [13]:
pyvista.set_jupyter_backend('client')
# Create plotter and pyvista grid
p = pyvista.Plotter()
topology, cell_types, geometry = plot.vtk_mesh(V_u)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)

# Attach vector values to grid and warp grid by vector
grid["u"] = uh.x.array.reshape((geometry.shape[0], 3))
#actor_0 = p.add_mesh(grid, style="wireframe", color="k")
warped = grid.warp_by_vector("u", factor =1)
warped.cell_data['mag'] = mises.vector.array /1e6
warped.set_active_scalars("mag")
actor_1 = p.add_mesh(warped, show_edges=True)
p.enable_point_picking()
p.show_axes()
if not pyvista.OFF_SCREEN:
    p.show()
else:
    figure_as_array = p.screenshot("deflection.png")



Widget(value="<iframe src='http://localhost:37921/index.html?ui=P_0xffff6c687580_0&reconnect=auto' style='widt…